# Deal with writing a csv file for all the reports - no distinction between train and test data; only for the unsupervised case

Author: Geeticka Chauhan

Please refer to the supervised version of this notebook for documentation related to similar components in both notebooks

Before using this, need to make sure that data conversion for the language modeling has been run to create the csv file from all reports and process them. 

In [317]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [486]:
%autoreload

import os
import sys
sys.path.append('../..')
import pandas as pd
import chestxray_joint.data.text.utils as utils
import chestxray_joint.data.text.lm_utils as lm_utils
import re
import chestxray_joint.data.text.preprocess as preprocess

allCHF_or_allCXR = "allCXR"

class_info_dir = '/data/vision/polina/projects/chestxray/work_space_v2/report_processing/edema_labels-12-03-2019/'
seth_info_dir = '/data/vision/polina/projects/chestxray/geeticka/class_information/' # class information and reports are located here
raw_reports_dir = '/data/vision/polina/projects/chestxray/data_v2/reports/'
out_dir = '/data/vision/polina/projects/chestxray/geeticka/pre-processed/semisupervised/' + allCHF_or_allCXR + '/'
# latest_csv = '/data/vision/polina/projects/chestxray/work_space_v2/report_processing/edema_labels-12-03-2019/mimic-cxr-sub-img-edema-split.csv'
latest_csv = '/data/vision/polina/projects/chestxray/work_space_v2/report_processing/edema_labels-12-03-2019/mimic-cxr-sub-img-edema-split-{}.csv'.format(allCHF_or_allCXR)
# this is where the outputted csv files will be located inside the directory reports_list/

def class_info_res(filename):
    return os.path.join(class_info_dir, filename)
def seth_labels_res(filename):
    return os.path.join(seth_info_dir, filename)
def raw_reports_res(filename):
    return os.path.join(raw_reports_dir, filename)
def out_res(filename): return os.path.join(out_dir, filename)
def lm_res(filename): return os.path.join('/data/vision/polina/projects/chestxray/geeticka/pre-processed/', filename)

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

### Following are the different classes with their keywords:
class0 None: 'no pulmonary edema', 'no vascular congestion', 'no fluid overload’, 'no acute cardiopulmonary    process’                                                                                                           

class1 Vascular congestion: 'mild pulmonary vascular congestion', ‘cephalization', 'mild hilar engorgement','mild vascular plethora'                                                                                                

class2 Interstitial edema: 'kerley', 'interstitial edema’, 'interstitial thickening’, 'interstitial opacities’

class3 Alveolar edema: 'perihilar infiltrates', 'peri-hilar infiltrates', 'hilar infiltrates', 'alveolar infiltrates', 'severe pulmonary edema’ 

In [366]:
edema_pred_dict = {'none': 0 ,'vascular congestion': 1, 'interstitial edema': 2, 'alveolar edema': 3}
# might be interesting to consider the ranking loss function for the classification task in the future
# this needs to be used when generating the test data csv

In [367]:
data = pd.read_csv(latest_csv)
def convert_to_filename(row):
    filename = row['study_id']
    study_id = 's'+ str(filename) + '.txt'
    return study_id
data['study_id'] = data.apply(convert_to_filename, axis=1)
# data_edema = data.loc[data['edeme_severity'] >= 0].reset_index(drop=True)

/data/vision/polina/shared_software/anaconda3-4.3.1/envs/chestxray_joint/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [368]:
data = data.drop_duplicates(subset=['study_id', 'edeme_severity'])

In [369]:
len(data) # this is just to get the reports, that's why the number is less than 17,000

222254

`data` is less because lateral views of some images is not available, so we lost some reports. For all CXR we have 224673 reports but above is less by 2000 reports

train.csv contains duplicates from test.csv - need to remove those

In [375]:
# this represents the supervised dataset that we have access to. The rest of the reports are unlabeled
train_df_filename = utils.train_filename_df(class_info_res) # this data is incomplete in terms of the keywords that 
# I have

In [376]:
len(train_df_filename)

6710

Above is the supervised part of the data, so we could find keywords there

In [377]:
# data_edema.rename(columns={'study_id':'filename',
#                           'Edema severity':'edema_severity'}, 
#                  inplace=True)

In [378]:
# data_edema = data_edema[['filename', 'edema_severity']]

In [379]:
# data_edema['metadata'] = '{}'
# train_df_filename = data_edema

Next todo: Need to write the test file and then double check if all the reports in the folder are located. Also check if there is any report overlap for train vs test. Also while writing the test file, need to make sure that s1232.txt appears rather than 1232.txt. 

## Write the train and test files

In [380]:
test_df_filename, duplicated_filenames = utils.test_filename_df(seth_labels_res, edema_pred_dict)

Radiologist (Seth) did not label report: s52375169.txt
Original test data frame is 199 long but there are 21 duplicated rows
Removed duplicates to return 178 length dataframe


In [381]:
test_filenames = test_df_filename['filename'].unique().tolist()

In [382]:
len(test_filenames)

178

In [383]:
train_filenames_overlap = train_df_filename[train_df_filename['filename'].isin(test_filenames)]['filename'].unique().tolist()
print("%d filenames from the test data are present in the train_filename.csv file. These will need to be removed"%(
len(train_filenames_overlap)))

178 filenames from the test data are present in the train_filename.csv file. These will need to be removed


In [384]:
set(test_filenames) - set(train_filenames_overlap)

set()

above set should be 0

#### Removing test filenames from the train data

In [385]:
train_df_filename = train_df_filename[~train_df_filename['filename'].isin(test_filenames)].reset_index(drop=True)

In [386]:
all_data_filename = pd.concat([train_df_filename, test_df_filename], axis=0, ignore_index=True)

In [387]:
supervised_filenames = list(all_data_filename['filename'])
data_unsupervised_edema = data[['study_id', 'edeme_severity']]
data_unsupervised_edema = data_unsupervised_edema[~data_unsupervised_edema['study_id'].isin(supervised_filenames)].reset_index(drop=True)

In [388]:
len(data_unsupervised_edema) + len(supervised_filenames)

222304

In [389]:
no_frontal_view_filenames = list(set(train_df_filename['filename']) - set(data['study_id']))
# looks like the CHF data split is missing 50 files that are present in the class files. 
# splits file needs to be fixed to incorporate these in
# above don't have any frontal view. Remove those. 
all_data_filename = all_data_filename[~all_data_filename['filename'].isin(no_frontal_view_filenames)].reset_index(drop=True)

In [390]:
len(all_data_filename) + len(data_unsupervised_edema)
# now this matches the number of files that we have in the splits file

222254

In [391]:
unsupervised_data_filename = data_unsupervised_edema.copy()
unsupervised_data_filename.rename(columns={'study_id':'filename', 'edeme_severity': 'edema_severity'}, inplace=True)
def return_blank_keywords(row):
    return {'keywords_found': []}
unsupervised_data_filename['metadata'] = unsupervised_data_filename.apply(return_blank_keywords, axis=1)
# unsupervised_data_filename['metadata'] = {'keywords_found': []}

In [392]:
all_data_filename = pd.concat([all_data_filename, unsupervised_data_filename], axis=0, ignore_index=True)

Therefore, `all_data` contains the labels of the train files from the keyword matching, but the test files from seth labels. This is necessary for the testing stage

## Write the train and test filename csv

In [393]:
if not(os.path.exists(out_res('reports_list'))):
    os.mkdir(out_res('reports_list'))

In [395]:
# utils.write_dataframe(all_data_filename, out_res('reports_list/all_data_list.csv'))
# # only write the above dataframe when the duplicates from the test data have been removed. 

# Now let's grab the actual text located in these and tokenize it. We will use scispacy to do the same. 

In [487]:
all_filenames = all_data_filename['filename'].unique().tolist()

In [488]:
len(all_filenames)

222254

In [489]:
# # take in a line in a report and do some pre-processing like number normalization, sentence segmentation etc
# def pre_process_report(line_report):
#     # do this later

In [399]:
original_reports_df = lm_utils.read_dataframe(lm_res('lm_reports/original_reports.csv'))

In [492]:
all_data_reports, files_without_sections = utils.write_report_into_df(all_data_filename, original_reports_df, semisupervised=True)
# this will append all content in final_report when nothing can be found for the findings and impressions

report s54168089.txt is empty


In [494]:
files_without_sections

10716

In [495]:
percentage = float(files_without_sections)/len(all_data_reports)*100

In [496]:
print("%.2f percent of the reports did not have finding, impression, conclusion and recommendation sections" %(percentage))

4.82 percent of the reports did not have finding, impression, conclusion and recommendation sections


In [497]:
len(all_data_reports)

222254

### Write the train and test files with original text

In [427]:
if not os.path.exists(out_res('reports')):
    os.mkdir(out_res('reports'))

In [429]:
# utils.write_dataframe(all_data_reports, out_res('reports/all_data.csv'))
# # # only write the above dataframe when the duplicates from the test data have been removed. 

In [430]:
# indexes = [2,10,18,34,40,48,55,61,63,70,72,73,74,76,88,101,106,119,147,148,155,159,165,169,170]
# for metadata in test_df_reports.iloc[indexes]['metadata']:
#     print(metadata['keywords_found'])
# #     print(report)
# #     print(report['metadata']['keywords_found'], '\n')

Now check if any of the filenames in the original csv that ray is using is not present in all_data_reports - that should not be the case

### Tokenize and pre-process the data

In [431]:
import spacy
import scispacy
nlp = spacy.load('en_core_sci_md')
# from scispacy.umls_linking import UmlsEntityLinker

In [432]:
# linker = UmlsEntityLinker()

In [433]:
# nlp.add_pipe(linker)

In [434]:
# punctuations = ['.', ',', ':', '?', '!', ';', '-', '(', ')', '{', '}', '"', "'"]
# def is_punct(char):
#     if char in punctuations:
#         return True
#     else:
#         return False

In [ ]:
all_data_reports['normalized_report'] = all_data_reports.apply(preprocess.normalize_report, axis=1)

## Write the reports with normalized data

In [ ]:
if not os.path.exists(out_res('reports_normalized')):
    os.mkdir(out_res('reports_normalized'))

In [ ]:
# utils.write_dataframe(all_data_reports, out_res('reports_normalized/all_data_original.csv'))
# # only write the above dataframe when the duplicates from the test data have been removed. 

In [ ]:
# utils.write_dataframe(test_df_reports, out_res('reports_normalized/test_original.csv'))
# # only write the above dataframe when the duplicates from the test data have been removed. 

## NOW, the next step is to use the script located in chestxray_joint/data/text/data_splitting.py to generate the tsv files

In [468]:
output_channel_encoding = 'multiclass' # or multilabel
training_mode = 'semisupervised' # or semisupervised

In [469]:
# TODO: fill this in order to do the generation of the tsv files as well as make the code more modular
tsv_in_dir = '/data/vision/polina/projects/chestxray/geeticka/pre-processed/' + training_mode \
+ '/' + allCHF_or_allCXR + '/reports_normalized'
tsv_out_dir = '/data/vision/polina/projects/chestxray/geeticka/bert/converted_data/' + output_channel_encoding + \
'/' + training_mode

if training_mode == 'semisupervised':
    tsv_out_dir += '/' + allCHF_or_allCXR

def tsv_in_res(filename): return os.path.join(tsv_in_dir, filename)
def tsv_out_res(filename): return os.path.join(tsv_out_dir, filename)

# development_or_test = 'development'

In [470]:
all_data_df = utils.read_dataframe(tsv_in_res('all_data_original.csv'))
# test_df = utils.read_dataframe(tsv_in_res('test_original.csv'))

In [ ]:
# all_data_df.fillna('', inplace=True) # there were some NaN values in original_report and normalized_report

In [471]:
if not os.path.exists(tsv_out_res('full')):
    os.makedirs(tsv_out_res('full'))

### Write the newly formed train and dev files (that are taken from original train data and are only to be used for tuning)

Need to figure out what to do about these empty reports now that we have the semi supervised case. The report extraction needs to be made more sophisticated, based on the label. If there is no label, then maybe extract the whole final_report section? 

In [472]:
len(all_data_df)

222254

In [473]:
all_data_df.loc[all_data_df['filename'] == 's56406919.txt']

,filename,edema_severity,original_report,metadata,normalized_report
184233,s56406919.txt,-1,"DEID -year-old male with aortic stenosis, cong...","{'keywords_found': [], 'origin_section': ['fin...",{'sentences': ['-year-old male with aortic ste...


In [476]:
for idx, row in all_data_df.iterrows():
#     print(row['filename'])
    if not utils.remove_whitespace(row['original_report']):
        print('%s is empty'%row['filename'])

s54168089.txt is empty


Looks like no report is empty anymore! In the future, can insert another flag for empty reports when reading the tsv file from the code in `chestxray_joint.model` and skip over the reports and images associated with those

In [477]:
new_all_data_df_bert = utils.get_df_bert(all_data_df, output_channel_encoding)
# new_all_data_df_bert.to_csv(tsv_out_res('full/all_data.tsv'), sep='\t', index=False, header=False)
# # don't need to keep rewriting these so can comment above out

In [ ]:
# new_dev_df_bert = utils.get_df_bert_multilabel(new_dev_df)
# new_dev_df_bert.to_csv(tsv_out_res('development/dev.tsv'), sep='\t', index=False, header=False)
# # don't need to keep rewriting these so can comment above out

### Write the original train and test files (that are to be used for reporting)

In [79]:
# train_df_bert = utils.get_df_bert_multilabel(train_df)
# train_df_bert.to_csv(tsv_out_res('testing/train.tsv'), sep='\t', index=False, header=False)

In [80]:
# test_df_bert = utils.get_df_bert_multilabel(test_df)
# test_df_bert.to_csv(tsv_out_res('testing/dev.tsv'), sep='\t', index=False, header=False)
# # we must call it dev for the purposes of the evaluation - that is just the name that the algorithm expects
# # this can probably be changed in the future

In [84]:
a = {1:'sd', 2:'sdas'}
list(a.keys())

[1, 2]